In [2]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [3]:
import numpy as np
import pandas as pd

import keras.backend as K
import keras
from keras.utils import plot_model

from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, TimeDistributed, LSTM, \
    Bidirectional, Flatten, \
    Embedding, Dropout, Flatten, BatchNormalization, \
    RNN, concatenate, Activation
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import random as rn
import tensorflow as tf


In [4]:
rn.seed(123)
np.random.seed(99)
tf.random.set_seed(1234)
# other parameters
MAX_SEQUENCE_LENGTH = 554
EMBEDDING_DIM = 300
nb_words = 3438

In [5]:
# load feature and labels
feat = np.load('/content/drive/MyDrive/2o Cuatri/TFG/scripts/npy/pAA-hsf.npy')
vad = np.load('/content/drive/MyDrive/2o Cuatri/TFG/scripts/npy/emotions.npy')

# remove outlier, < 1, > 5
vad = np.where(vad == 5.5, 5.0, vad)
vad = np.where(vad == 0.5, 1.0, vad)

print(f'feat shape: {feat.shape}')
print(f'vad shape: {vad.shape}')

feat shape: (10039, 68)
vad shape: (10039, 3)


In [6]:
# text feature
x_train_text = np.load(f'/content/drive/MyDrive/2o Cuatri/TFG/scripts/npy/text_seq_{MAX_SEQUENCE_LENGTH}.npy')
g_word_embedding_matrix = np.load(f'/content/drive/MyDrive/2o Cuatri/TFG/scripts/npy/embeddings.npy')

print(f'x_train_text shape: {x_train_text.shape}')
print(f'embeddings shape: {g_word_embedding_matrix.shape}')

x_train_text shape: (10039, 554)
embeddings shape: (3438, 300)


In [7]:
scaler = StandardScaler()
scaler = scaler.fit(feat)
scaled_feat = scaler.transform(feat)
feat = scaled_feat

In [8]:
scaler = MinMaxScaler(feature_range=(-1, 1))
# .reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
scaler = scaler.fit(vad)
# .reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
scaled_vad = scaler.transform(vad)
vad = scaled_vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold = K.squeeze(gold, axis=-1)
    pred = K.squeeze(pred, axis=-1)
    gold_mean = K.mean(gold, axis=-1, keepdims=True)
    pred_mean = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var = K.mean(K.square(gold-gold_mean), axis=-1, keepdims=True)
    pred_var = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc = K.constant(2.) * covariance / (gold_var + pred_var +
                                         K.square(gold_mean - pred_mean))
    #K.print_tensor(ccc)
    return ccc


def ccc_loss(gold, pred):
    # input (num_batches, seq_len, 1)
    ccc_loss = K.constant(1.) - ccc(gold, pred)
    return ccc_loss



In [9]:
def create_model(alpha, beta, gamma):
    # speech network
    input_speech = Input(shape=(1, feat.shape[1]), name='speech_input')
    net_speech = BatchNormalization()(input_speech)
    net_speech = LSTM(256, return_sequences=True)(net_speech)
    net_speech = LSTM(256, return_sequences=True)(net_speech)
    net_speech = LSTM(256, return_sequences=True)(net_speech)
    net_speech = Flatten()(net_speech)
    model_speech = Dropout(0.3)(net_speech)

    # text network
    input_text = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net_text = Embedding(nb_words,
                         EMBEDDING_DIM,
                         weights=[g_word_embedding_matrix],
                         trainable=True)(input_text)
    net_text = LSTM(256, return_sequences=True)(net_text)
    net_text = LSTM(256, return_sequences=True)(net_text)
    net_text = LSTM(256, return_sequences=False)(net_text)
    net_text = Dense(64)(net_text)
    model_text = Dropout(0.3)(net_text)
    

    # combined model
    model_combined = concatenate([model_speech, model_text])
    model_combined = Dense(64, activation='relu')(model_combined)
    model_combined = Dense(32, activation='relu')(model_combined)
    model_combined = Dropout(0.4)(model_combined)
    target_names = ('v', 'a', 'd')
    # model_combined = [Dense(1, name=name)(model_combined)
    #                   for name in target_names]
    model_valence = Dense(1, name='v')(model_combined)
    model_arousal = Dense(1, name='a')(model_combined)
    model_dominance = Dense(1, name='d')(model_combined)

    model_combined = [model_valence, model_arousal, model_dominance]
    ####### borrar esto ######
    # optimizer = keras.optimizers.RMSprop(clipnorm=2.0)
    #########################

    model = Model([input_speech, input_text], model_combined)
    model.compile(loss=ccc_loss,
                  loss_weights={'v': alpha, 'a': beta, 'd': gamma},
                  optimizer='rmsprop', metrics=[ccc])
    return model

In [10]:
import matplotlib.pyplot as plt
def plot_training(history):
  # summarize history for ccc
  plt.figure(figsize=(10,10))
  plt.plot(history.history['v_ccc'], color='green', label='v_ccc')
  plt.plot(history.history['a_ccc'], color='blue', label='a_ccc')
  plt.plot(history.history['d_ccc'], color='red', label= 'd_ccc')
  plt.title(f'Model Train CCC')
  plt.ylabel('ccc')
  plt.xlabel('epoch')
  plt.legend()
  plt.show()
  
  plt.figure(figsize=(10,10))
  plt.plot(history.history['val_v_ccc'], color='green', label= 'val_v_ccc')
  plt.plot(history.history['val_a_ccc'], color='blue', label= 'val_a_ccc')
  plt.plot(history.history['val_d_ccc'], color='red', label= 'val_d_ccc')
  plt.title(f'Model Validation CCC')
  plt.ylabel('ccc')
  plt.xlabel('epoch')
  plt.legend()
  plt.show()

  # summarize history for loss
  plt.figure(figsize=(10,10))
  plt.plot(history.history['v_loss'], color='green', label='v_loss')
  plt.plot(history.history['a_loss'], color='blue', label='a_loss')
  plt.plot(history.history['d_loss'], color='red', label= 'd_loss')
  plt.title(f'Model Loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend()
  plt.show()

  plt.figure(figsize=(10,10))
  plt.plot(history.history['val_v_loss'], color='green', label= 'val_v_loss')
  plt.plot(history.history['val_a_loss'], color='blue', label= 'val_a_loss')
  plt.plot(history.history['val_d_loss'], color='red', label= 'val_d_loss')
  plt.title(f'Model Validation Loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend()
  plt.show()

In [12]:
results = []
histories = []
for i in range(1):
  print(f'Iteration {i}\n')
  model = create_model(0.7, 0.2, 0.1)
  plot_model(model, show_shapes=True)

  # 7869 first data of session 5 (for LOSO)
  earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=15, # change: patience from 10 to 15
                            restore_best_weights=True)
  hist = model.fit([feat.reshape(10039, 1, 68)[:7869], x_train_text[:7869]],
                  {"v": vad[:7869].T[0], "a": vad[:7869].T[1],
                      "d": vad[:7869].T[2]},
                  batch_size=256,  # best:8
                  validation_split=0.2, epochs=6, verbose=1, shuffle=True,
                  callbacks=[earlystop])
  metrik = model.evaluate(
      [feat.reshape(10039, 1, 68)[7869:], x_train_text[7869:]],
      {"v": vad[7869:].T[0], "a": vad[7869:].T[1], "d": vad[7869:].T[2]})
  print("CCC: ", metrik[-3:])  # np.mean(metrik[-3:]))
  print("CCC_mean: ", np.mean(metrik[-3:]))
  results.append(metrik[-3:])
  histories.append(hist)
  # plot_training(hist)

# visualize trainig
ccc_list = [np.mean(r) for r in results]
v_list = [r[0] for r in results]
a_list = [r[1] for r in results]
d_list = [r[2] for r in results]

print(f'ccc_mean: {np.mean(ccc_list)} +- {np.std(ccc_list)} \n \
        valence_mean: {np.mean(v_list)} +- {np.std(v_list)} \n \
        arousal_mean: {np.mean(a_list)} +- {np.std(a_list)} \n \
        dominance_mean: {np.mean(d_list)} +- {np.std(d_list)} \n' )

print(f'Plot of best model: Model {np.argmax(ccc_list)} \t ccc_mean = {ccc_list[np.argmax(ccc_list)]}\t {results[np.argmax(ccc_list)]}')
#plot_training(histories[np.argmax(ccc_list)])

Iteration 0

Epoch 1/6
25/25 [==============================] - 57s 688ms/step - loss: 0.8206 - v_loss: 0.8612 - a_loss: 0.6810 - d_loss: 0.8151 - v_ccc: 0.1388 - a_ccc: 0.3190 - d_ccc: 0.1849 - val_loss: 0.5807 - val_v_loss: 0.6277 - val_a_loss: 0.4310 - val_d_loss: 0.5511 - val_v_ccc: 0.3723 - val_a_ccc: 0.5690 - val_d_ccc: 0.4489
Epoch 2/6
25/25 [==============================] - 15s 611ms/step - loss: 0.5088 - v_loss: 0.5260 - a_loss: 0.4278 - d_loss: 0.5505 - v_ccc: 0.4740 - a_ccc: 0.5722 - d_ccc: 0.4495 - val_loss: 0.4901 - val_v_loss: 0.5149 - val_a_loss: 0.3963 - val_d_loss: 0.5046 - val_v_ccc: 0.4851 - val_a_ccc: 0.6037 - val_d_ccc: 0.4954
Epoch 3/6
25/25 [==============================] - 16s 628ms/step - loss: 0.4291 - v_loss: 0.4306 - a_loss: 0.3782 - d_loss: 0.5205 - v_ccc: 0.5694 - a_ccc: 0.6218 - d_ccc: 0.4795 - val_loss: 0.5595 - val_v_loss: 0.6099 - val_a_loss: 0.4007 - val_d_loss: 0.5245 - val_v_ccc: 0.3901 - val_a_ccc: 0.5993 - val_d_ccc: 0.4755
Epoch 4/6
25/25 [====

In [11]:
# loss: 0.6016 - v_loss: 0.6783 - a_loss: 0.3834 - d_loss: 0.5012 - v_ccc: 0.3217 - a_ccc: 0.6166 - d_ccc: 0.4988
# loss: 0.4230 - v_loss: 0.4239 - a_loss: 0.3761 - d_loss: 0.5111 - v_ccc: 0.5761 - a_ccc: 0.6239 - d_ccc: 0.4889
vccc = 0 + 0 + -0.03707017 + -0.0659868
vccc/4

-0.0257642425